In [1]:
import pandas as pd
import re
import numpy as np
from sklearn.preprocessing import scale

In [3]:
df = pd.read_csv('final_wr.csv')

In [23]:
test_df = df[(df.rookie_season == 2013)]
test_df.drop('Unnamed: 0', axis = 1, inplace = True)
print test_df.shape
test_df.head()

(86, 64)


/Users/TerryONeill/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,name,season,age,draft_pos,team,weight,bmi,games,rush_atts,rush_yds,...,rookie_season,height_inches,start_ratio,dpis_drawn,dpi_yards,pct_team_tgts,pct_team_receptions,pct_of_team_passyards,pct_team_touchdowns,years_in_league
50,Jaron Brown,2013.0,23.0,UDFA,ARI,205.0,26.3,16.0,0.0,0.0,...,2013.0,74.0,0.000,0.0,0.0,0.029617,0.030303,0.034983,0.041667,0.0
53,Jaron Brown,2014.0,24.0,UDFA,ARI,205.0,26.3,16.0,0.0,0.0,...,2013.0,74.0,0.125,0.0,0.0,0.053691,0.065476,0.059342,0.090909,1.0
56,Jaron Brown,2015.0,25.0,UDFA,ARI,205.0,26.3,16.0,0.0,0.0,...,2013.0,74.0,0.000,1.0,32.0,0.035714,0.027431,0.027848,0.025641,2.0
103,Darius Johnson,2013.0,22.0,UDFA,ATL,175.0,25.1,10.0,0.0,0.0,...,2013.0,70.0,0.200,2.0,38.0,0.065250,0.049438,0.049493,0.038462,0.0
214,Robert Woods,2013.0,21.0,2-41,BUF,201.0,26.5,14.0,2.0,16.0,...,2013.0,73.0,1.000,1.0,6.0,0.162835,0.133779,0.189172,0.187500,0.0


In [28]:
def feat_engineering(data):
    ## adding in the drop constant
    data['dropK'] = np.log(data['drops'] +1)
    
    ## adding in the yards after catch constant
    data['yacK'] = data.yac*(data.yac/data.rec_yards)
    
    ## compiling the base function score
    data['base'] = (((data.rec_yards+data.yacK+data.dpi_yards+(data.DYAR*100))*(data.receptions+(data.first_down_ctchs*data.first_down_ctchpct)+((data.recs_ovr_25**2)/data.receptions)))/(data.fumbles+data.dropK + (data.targets/data.pct_team_tgts))**2)
    
    ## compiling td_points score
    data['td_points'] = (((data.rec_tds+data.rush_tds)/np.average(data.rec_tds+data.rush_tds))*data.pct_team_touchdowns)
    
    ## creating final compilation score column
    data['compilation'] = (data.base*100) + (data.td_points*7)
    
    ## We know the nulls are all coming from if a player has zero recieving yards so you 
    ## cannot divide by zero and you get a null value. So we are fine putting a zero here
    data.yacK.fillna(value = 0, inplace = True)
    
    return data
    
    

In [30]:
feat_engineering(test_df)

/Users/TerryONeill/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Users/TerryONeill/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/TerryONeill/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

,name,season,age,draft_pos,team,weight,bmi,games,rush_atts,rush_yds,...,pct_team_tgts,pct_team_receptions,pct_of_team_passyards,pct_team_touchdowns,years_in_league,dropK,yacK,base,td_points,compilation
50,Jaron Brown,2013.0,23.0,UDFA,ARI,205.0,26.3,16.0,0.0,0.0,...,0.029617,0.030303,0.034983,0.041667,0.0,0.000000,9.778571,0.007400,0.022537,0.897773
53,Jaron Brown,2014.0,24.0,UDFA,ARI,205.0,26.3,16.0,0.0,0.0,...,0.053691,0.065476,0.059342,0.090909,1.0,0.693147,17.886463,0.023728,0.098342,3.061225
56,Jaron Brown,2015.0,25.0,UDFA,ARI,205.0,26.3,16.0,0.0,0.0,...,0.035714,0.027431,0.027848,0.025641,2.0,0.000000,4.340278,0.007066,0.013869,0.803711
103,Darius Johnson,2013.0,22.0,UDFA,ATL,175.0,25.1,10.0,0.0,0.0,...,0.065250,0.049438,0.049493,0.038462,0.0,1.386294,38.571429,0.016087,0.020803,1.754363
214,Robert Woods,2013.0,21.0,2-41,BUF,201.0,26.5,14.0,2.0,16.0,...,0.162835,0.133779,0.189172,0.187500,0.0,0.693147,24.531516,0.131703,0.304245,15.300014
215,Chris Hogan,2013.0,25.0,UDFA,BUF,220.0,29.0,16.0,0.0,0.0,...,0.032567,0.033445,0.026748,0.000000,0.0,0.000000,10.843373,0.003271,0.000000,0.327132
217,Marquise Goodwin,2013.0,23.0,3-78,BUF,183.0,27.0,12.0,3.0,13.0,...,0.061303,0.056856,0.091202,0.187500,0.0,0.693147,8.833922,0.037605,0.304245,5.890185
221,Chris Hogan,2014.0,26.0,UDFA,BUF,220.0,29.0,16.0,0.0,0.0,...,0.105354,0.112948,0.117875,0.173913,1.0,1.098612,91.100939,0.081711,0.376265,10.805001
222,Robert Woods,2014.0,22.0,2-41,BUF,201.0,26.5,16.0,0.0,0.0,...,0.179620,0.179063,0.193414,0.217391,1.0,1.791759,66.130186,0.197234,0.587914,23.838811
223,Robert Woods,2015.0,23.0,2-41,BUF,201.0,26.5,14.0,1.0,0.0,...,0.172043,0.159322,0.165121,0.130435,2.0,1.098612,34.001812,0.164613,0.211649,17.942832


In [46]:
def pivot_df(df):
    ## I am creating a list of the columns where there are no null values in order to create
    ## my pivot table with it

    cols_to_keep = []
    for col in df.columns:
        if df[col].isnull().sum() == 0:
            cols_to_keep.append(col)
    
    ## I am removing some columns that I do not want to be used
    cols_to_keep.remove('years_in_league')
    cols_to_keep.remove('team_pass_yds')
    cols_to_keep.remove('team_pass_tds')
    cols_to_keep.remove('team_pass_attempts')
    cols_to_keep.remove('team_completions')
    cols_to_keep.remove('total_points')
    cols_to_keep.append('compilation')
    
    ## creating a pivot table with columns that have no nulls in it and name as the index, with
    ## years in the league as the breakdown for columns
    pivoted = df.pivot_table(index=df['name'], columns='years_in_league', values=cols_to_keep)
    
    ## below we will lay out how we want to fill any null values for players
    ## that might not have been playing for the three initial seasons
    zero_cols = ['games', 'rush_atts', 'rush_yds', 'rush_y/a', 'rush_tds', 'rush_ypg',
             'targets', 'receptions', 'rec_yards', 'yards/reception', 'rec_tds',
             'rec_ypg', 'ctch_pct', 'y/tgt', 'fumbles', 'fumbles_recovered', 'fum_ret_yds',
             'fum_tds', 'forced_fumbles', 'pro_bowls', 'all_pros', '100yd_gms',
             'first_down_ctchs', 'first_down_ctchpct', 'long_ctch', 'drops', 'EYds',
             'DVOA', 'DYAR', '40 Yard', 'start_ratio', 'dpis_drawn', 'dpi_yards',
             'pct_team_tgts', 'pct_team_receptions', 'pct_of_team_passyards',
             'pct_team_touchdowns', 'dropK', 'yacK', 'td_points', 'compilation']

    backfill_cols = ['weight', 'bmi', 'rookie_age',
                     'rookie_season', 'height_inches']

    team_cols = ['team_pass_tds', 'team_pass_yds', 'team_pass_attempts', 'team_completions',
                 'total_points']

    years = [0.0, 1.0, 2.0]
    back_years = [1.0, 2.0]
    
    for col in zero_cols:
        for i in years:
            pivoted[col][i].fillna(0, inplace = True)
    
    for col in backfill_cols:
        for i in back_years:
            pivoted[col][i] = pivoted[col][0.0]
            
    pivoted = pivoted[pivoted.season[0.0].isnull() == False]
    
    ## creating a variable made of the multiindexed columns
    mi = pivoted.columns
    
    ## I am now creating a new set of column names made up of the upper index and the lower
    ## index added together
    new_cols = pd.Index([x[0]+'_'+str(x[1]) for x in mi.tolist()])
    
    ## I will now be renaming the pivoted dataframe columns my new_cols list which is a combination
    ## of the two levels of column names from when we initially pivoted
    pivoted.columns = new_cols
    print pivoted.columns
    pivoted.head()
    
    ## this is simply to get rid of the '.0' at the end of every column name
    pivoted.rename(columns = lambda x: x.replace('.0', ''), inplace = True)
    
    ## here I am correcting the values for season 1, 2
    pivoted['season_1'] = pivoted['season_0']+1
    pivoted['season_2'] = pivoted['season_0']+2
    
    ## here i am correcting the values for age 1 and 2
    pivoted['age_1'] = pivoted['age_0']+1
    pivoted['age_2'] = pivoted['age_0']+2
    
    return pivoted

In [47]:
pivot_df(test_df)

/Users/TerryONeill/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Index([u'season_0.0', u'season_1.0', u'season_2.0', u'age_0.0', u'age_1.0',
       u'age_2.0', u'weight_0.0', u'weight_1.0', u'weight_2.0', u'bmi_0.0',
       ...
       u'dropK_2.0', u'yacK_0.0', u'yacK_1.0', u'yacK_2.0', u'td_points_0.0',
       u'td_points_1.0', u'td_points_2.0', u'compilation_0.0',
       u'compilation_1.0', u'compilation_2.0'],
      dtype='object', length=144)


,season_0,season_1,season_2,age_0,age_1,age_2,weight_0,weight_1,weight_2,bmi_0,...,dropK_2,yacK_0,yacK_1,yacK_2,td_points_0,td_points_1,td_points_2,compilation_0,compilation_1,compilation_2
name,,,,,,,,,,,,,,,,,,,,,
Aaron Dobson,2013.0,2014.0,2015.0,22.0,23.0,24.0,210.0,210.0,210.0,26.2,...,0.693147,82.560694,12.736842,10.241135,0.332850,0.000000,0.000000,10.546819,0.083563,0.966747
Ace Sanders,2013.0,2014.0,2015.0,22.0,23.0,24.0,173.0,173.0,173.0,27.1,...,0.000000,189.688017,11.363636,0.000000,0.033805,0.000000,0.000000,12.428353,0.229302,0.000000
Ben Watson,2013.0,2014.0,2015.0,33.0,34.0,35.0,255.0,255.0,255.0,31.9,...,0.000000,0.000000,0.000000,0.000000,0.052769,0.000000,0.000000,0.000000,0.000000,0.000000
Chris Hogan,2013.0,2014.0,2015.0,25.0,26.0,27.0,220.0,220.0,220.0,29.0,...,1.609438,10.843373,91.100939,26.402222,0.000000,0.376265,0.094066,0.327132,10.805001,10.623025
Cordarrelle Patterson,2013.0,2014.0,2015.0,22.0,23.0,24.0,216.0,216.0,216.0,27.7,...,0.000000,180.556503,71.760417,0.900000,0.841370,0.063633,0.000000,19.582420,7.735541,0.063590
Da'Rick Rogers,2013.0,2014.0,2015.0,22.0,23.0,24.0,216.0,216.0,216.0,27.7,...,0.000000,72.520833,0.000000,0.000000,0.074604,0.000000,0.000000,2.289282,0.000000,0.000000
Darius Johnson,2013.0,2014.0,2015.0,22.0,23.0,24.0,175.0,175.0,175.0,25.1,...,0.000000,38.571429,0.000000,0.000000,0.020803,0.000000,0.000000,1.754363,0.000000,0.000000
David Johnson,2013.0,2014.0,2015.0,26.0,27.0,28.0,270.0,270.0,270.0,34.7,...,0.000000,17.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.315610,0.000000,0.000000
DeAndre Hopkins,2013.0,2014.0,2015.0,21.0,22.0,23.0,214.0,214.0,214.0,28.2,...,1.386294,43.602244,123.776033,19.677186,0.113870,0.885077,2.256777,21.296900,86.887206,92.352261


# Performing the LDA

In [82]:
train_df = pd.read_csv('/Users/TerryONeill/Terry_git/Capstone/GABBERT/wide_receivers/pivot_catcherr.csv')
train_df.drop('Unnamed: 0', axis = 1, inplace = True)

comp_df_cols = ['name', 'compilation_3']

comp_df = pd.DataFrame(train_df[comp_df_cols], columns = comp_df_cols)
train_df.set_index(train_df.name, drop = True, inplace = True)
train_df.drop('name', axis = 1, inplace = True)
train_df.head()

,season_0,season_1,season_2,age_0,age_1,age_2,weight_0,weight_1,weight_2,bmi_0,...,td_points_2,compilation_0,compilation_1,compilation_2,compilation_3,avg_starts,dpis,dpi_yards,year_1_growth,year_2_growth
name,,,,,,,,,,,,,,,,,,,,,
A.J. Green,2011.0,2012.0,2013.0,23.0,24.0,25.0,205.0,205.0,205.0,25.0,...,1.562539,53.006410,98.471570,82.935491,59.741763,1.000000,0.0,0.0,9.264454,-13.711468
A.J. Jenkins,2012.0,2013.0,2014.0,23.0,24.0,25.0,192.0,192.0,192.0,26.0,...,0.000000,0.000000,0.597268,0.000000,0.000000,0.020833,0.0,0.0,0.121706,-0.527123
Aaron Brown,2009.0,2010.0,2011.0,24.0,25.0,26.0,196.0,196.0,196.0,26.6,...,0.000000,1.007213,0.000000,0.000000,0.000000,0.044444,0.0,0.0,-0.205240,0.000000
Aaron Halterman,2007.0,2008.0,2009.0,25.0,26.0,27.0,255.0,255.0,255.0,30.2,...,0.000000,0.030196,0.000000,0.000000,0.000000,0.000000,0.0,0.0,-0.006153,0.000000
Aaron Moorehead,2003.0,2004.0,2005.0,23.0,24.0,25.0,200.0,200.0,200.0,25.0,...,0.000000,0.240777,0.025880,0.222518,0.274562,0.000000,0.0,0.0,-0.043790,0.173544


In [83]:
# Create an average starts column
train_df['avg_starts'] = (train_df.start_ratio_0 + train_df.start_ratio_1 + train_df.start_ratio_2) / 3

#Create a column that adds up a player's dpi yards and penaltys drawn
train_df['dpis'] = train_df.dpis_drawn_0 + train_df.dpis_drawn_1 + train_df.dpis_drawn_2
train_df['dpi_yards'] = train_df.dpi_yards_0 + train_df.dpi_yards_1 + train_df.dpi_yards_2

In [84]:
## this is a  list of features without any first year stats in it as they
## have been found to be not very indicative of the target
features_no_year_1 = ['age_2', 'weight_2', 'bmi_2',
         'rush_y/a_1', 'rush_y/a_2',
         'receptions_1', 'receptions_2',
        'rec_yards_1','rec_yards_2', 'rec_tds_1',
        'rec_tds_2', 'ctch_pct_1', 'ctch_pct_2',
         'first_down_ctchpct_1',
        'first_down_ctchpct_2',  'long_ctch_1', 'long_ctch_2',
         'drops_1', 'drops_2',  'EYds_1', 'EYds_2',
        'DVOA_1', 'DVOA_2', 'height_inches_2', 'avg_starts', 'dpis', 'dpi_yards',
         'pct_team_tgts_1',
        'pct_team_tgts_2', 'compilation_0', 'compilation_1', 'compilation_2', 'yacK_2',
                 'year_1_growth', 'year_2_growth']

## creating dataframe to perform LDA on
lda_df = train_df[features_no_year_1]


In [85]:
# Create categories for player season_3 ratings

bins = [-1, 10, 30, 65, 200]
labels = ['below average', 'league_average', 'quality starter', 'all_pro']
train_df['categories'] =  pd.cut(train_df['compilation_3'], bins, labels=labels)

In [86]:
from sklearn.lda import LDA

lda = LDA(n_components=4)

X = scale(train_df.drop(['compilation_3', 'categories'], axis = 1))
y = train_df['categories']

## fit and transform the standardized data
lda_cols = lda.fit_transform(X, y)

In [87]:
lda_df = pd.DataFrame(lda_cols, columns = ['lda1', 'lda2', 'lda3'])


In [88]:
lda_df.set_index(train_df.index, drop = False, inplace = True)
lda_df.head()

,lda1,lda2,lda3
name,,,
A.J. Green,-10.677512,4.379239,-3.186913
A.J. Jenkins,1.382989,0.775394,-0.357371
Aaron Brown,1.541675,0.066295,0.245114
Aaron Halterman,1.538582,0.094431,0.520712
Aaron Moorehead,1.190515,-0.061401,-0.911344
